In [2]:
import json
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from hummingbird.ml import convert
import torch
import ezkl
import os



iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = MLPClassifier()
clr.fit(X_train, y_train)

circuit = convert(clr, "torch", X_test[:1]).model


/home/lexx/code/ezkl_test/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [4]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')


In [5]:
# export to onnx format
# !!!!!!!!!!!!!!!!! This will flash a warning but it is fine !!!!!!!!!!!!!!!!!!!!!

# Input to the model
shape = X_train.shape[1:]
x = torch.rand(1, *shape, requires_grad=True)
torch_out = circuit(x)
# Export the model
torch.onnx.export(circuit,               # model being run
                  # model input (or a tuple for multiple inputs)
                  x,
                  # where to save the model (can be a file or file-like object)
                  "network.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})

d = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_shapes=[shape],
            input_data=[d],
            output_data=[((o).detach().numpy()).reshape([-1]).tolist() for o in torch_out])

# Serialize data into file:
json.dump(data, open("input.json", 'w'))


In [6]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True


In [7]:
cal_path = os.path.join("calibration.json")

data_array = (torch.rand(20, *shape, requires_grad=True).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")


Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns 

True

In [8]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True


In [9]:
# srs path
res = await ezkl.get_srs( settings_path)


In [10]:
# now generate the witness file 

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)


In [11]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)


In [12]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)


{'instances': [['0200000000000000000000000000000000000000000000000000000000000000', 'ee01000000000000000000000000000000000000000000000000000000000000', 'c402000000000000000000000000000000000000000000000000000000000000', '4e03000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x211eac21015f6bcdf19dde03600d5a3b68c46bbb6658b73b0efc2697061403b81940ebd0d1c1a46346049e07f5eca218352aecf721689f29530abd12d077395814059105c54d5bc9f258ada43b660d69c4397a43870f075566b72090a8659e4713c8972f67cfa90223380a92e23b0de2fee2cd7c7a135bb9670f84d5811287ed104641a2a1e2b0e5c5ae0b6aff1fb1791c9a9d2dd694a37200f163775312a0e3105642a3342d814107fd3a8050c54a36f4e632422e4fac65da7898080354686a108f1e58a283161a46ffef5a956b38ebf88c04fbc85797b170c1a4cb1f6e2c92086eed40a935903e8da74a4be7e3fe7b33dc516062a2a5ba225d7bf95f8090df062ad6ddc0301e07f9e39fca130281c9a38d319d735c7b36cadec88d66bb7f2d2a039e1b21d40c3655c0bad76c505394347f80b28feabac2d1d6eab09babe29b0024af35c03ddfea9f9c46b3b95321cb9e0198db651e2f3c97cc14d6b3

In [13]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")

verified


In [3]:
# export to onnx format
# !!!!!!!!!!!!!!!!! This will flash a warning but it is fine !!!!!!!!!!!!!!!!!!!!!

for i in range(3000):
  # Input to the model
  shape = X_train.shape[1:]
  x = torch.rand(1, *shape, requires_grad=True)
  torch_out = circuit(x)
  # Export the model
  # torch.onnx.export(circuit,               # model being run
  #                   # model input (or a tuple for multiple inputs)
  #                   x,
  #                   # where to save the model (can be a file or file-like object)
  #                   "network.onnx",
  #                   export_params=True,        # store the trained parameter weights inside the model file
  #                   opset_version=10,          # the ONNX version to export the model to
  #                   do_constant_folding=True,  # whether to execute constant folding for optimization
  #                   input_names=['input'],   # the model's input names
  #                   output_names=['output'],  # the model's output names
  #                   dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
  #                                 'output': {0: 'batch_size'}})

  d = ((x).detach().numpy()).reshape([-1]).tolist()

  data = dict(input_shapes=[shape],
              input_data=[d],
              output_data=[((o).detach().numpy()).reshape([-1]).tolist() for o in torch_out])

  # Serialize data into file:
  json.dump(data, open(f"test_inputs/input_{i}.json", 'w'))
